This is the notebook of the following medium difficulty task: (OpenCV) Try heuristic algorithms e.g., you select an image of the place without people. You subtract all images from that image in your test set. The number of shapes larger than a threshold size are people. Compare speed/performance. Feel free to suggest other heuristics.

We are basically trying to look at the difference between the background image and the image filled with people. This way we can detect the people in an image.

In [2]:
import cv2
import os
import PIL
import matplotlib


# Load the background image 1666696481740.jpg from ordered_data\0\1666696481740.jpg
#bg = cv2.imread("ordered_data/0/1666696985773.jpg")
bg = cv2.imread("ordered_data/0/1666711469734.jpg")
# Set the path to the directory containing the images
path = "data/1 Oude Fitness/partition 5"

# Iterate over the images in the directory
for filename in os.listdir(path):
    # Load the image
    img = cv2.imread(os.path.join(path, filename))


    # Subtract the background image from the current image
    diff = cv2.absdiff(bg, img)

    # Convert the resulting image to grayscale
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    # print the gray image
    cv2.imshow("Gray", gray)
    PIL.Image.fromarray(gray).save("gray.jpg")

    # Threshold the image to create a binary image with the moving objects
    _, thresh = cv2.threshold(gray, 80, 255, cv2.THRESH_BINARY)

    # Find the contours of the moving objects
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours
    for c in contours:
        # Check if the contour is larger than a threshold size (e.g., 100 pixels)
        if cv2.contourArea(c) > 100:
            # Draw a rectangle around the contour
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the original frame with the moving objects highlighted
    cv2.imshow("Frame", img)
    cv2.waitKey(0)

# Close the window
cv2.destroyAllWindows()

When run, the script will display each image in a separate Python window and highlight any people detected with a green bounding box. 

#### the user can press the "0" key to move on to the next image and observe the algorithm's detections.
In terms of performance, this algorithm is effective in detecting people, but may struggle with variations in lighting conditions. It may mark objects incorrectly when there are significant differences in lighting between the input image and the original image used as a reference. 

In [1]:
import cv2
import os
import time

# Load the HOG + SVM based detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# Set the path to the directory containing the images
path = "data/1 Oude Fitness/partition 5"

# Iterate over the images in the directory
for filename in os.listdir(path):
    # Load the image
    start_time = time.time()
    image = cv2.imread(os.path.join(path, filename))

    # Perform the detection
    boxes, weights = hog.detectMultiScale(image, winStride=(8,8), hitThreshold = 0)


    # Draw the bounding boxes on the image
    for (x, y, w, h) in boxes:
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)


    # Show the image with the bounding boxes
    cv2.imshow("Image", image)
    cv2.waitKey(0)
    


# Close the window
cv2.destroyAllWindows()


The HOG + SVM based detector algorithm is well-suited for handling variations in lighting conditions. However, it may come at the cost of reduced overall accuracy and fewer detections of people in the images. Despite this, the algorithm is able to maintain a high level of precision by making very few to no errors in light variations. This algorithm is specialized for detecting people.

In [1]:
import cv2
import os

# Create a MOG2 background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2()

# Set the path to the directory containing the images
path = "data/1 Oude Fitness/partition 5"

# Iterate over the images in the directory
for filename in os.listdir(path):
    # Load the image
    img = cv2.imread(os.path.join(path, filename))

    # Use the MOG2 background subtractor to get the foreground mask
    fgmask = fgbg.apply(img)

    # Threshold the foreground mask to create a binary image with the moving objects
    _, thresh = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)

    # Find the contours of the moving objects
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Iterate over the contours
    for c in contours:
        # Check if the contour is larger than a threshold size (e.g., 100 pixels)
        if cv2.contourArea(c) > 100:
            # Draw a rectangle around the contour
            (x, y, w, h) = cv2.boundingRect(c)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    # Show the original frame with the moving objects highlighted
    cv2.imshow("Frame", img)
    cv2.waitKey(0)

# Close the window
cv2.destroyAllWindows()

unlike the previous code MOG2 is not limited to detecting people. It is trained on detecting moving objects. So this would be a good fit to detect any object but not people persee. 

#### SPEED: To measure the performance, remove the commenting from "cv2.waitKey(0)" in each algorithm to allow the code to execute without interruption."

#### In terms of speed, algorithm 1 and 2 seem to perform quite good! The second algorithm HOG + SVM takes longer because it is performing a more complex detection task. 